https://www.youtube.com/watch?v=pSexXMdruFM&list=PLZbbT5o_s2xrfNyHZsM6ufI0iZENK9xgG&index=29

- Tensorboard is Tensorflows machine larning toolkit.
- It is a frotend web interface.
- It reads from a file, and we can do that(writing) using pytorch summary writer.
- It allows us to 
  - track and visualize our metrics, like our loss and accuracy
  - histograms of weights and biases etc
  - visualize our networks graph

### Normal way in command line:
- !tensorboard --version
- tensorboard --logdir=runs
- this gives a localhost, where tensorboard will be running
- `runs` is a folder in the current directory

### Other way(colab):
https://medium.com/p/16b4bb9812a6/responses/show

In [0]:
!pip install tensorboardcolab

In [0]:
from tensorboardcolab import TensorBoardColab
tbc=TensorBoardColab(graph_path='./runs')

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://445fca16.ngrok.io


In [0]:
??TensorBoardColab

## The Neural Network

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

batch_size = 128

import numpy as np
import matplotlib.pyplot as plt
torch.set_printoptions(linewidth=120)

from torch.utils.tensorboard import SummaryWriter

In [4]:
print(torch.__version__)
print(torchvision.__version__)

1.1.0
0.3.0


In [0]:
def get_num_correct(preds,labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [0]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6 , out_channels=12 , kernel_size=5)
        #we increase the number of output channels

        self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
        ## number of output channels * h * w of output channel
        ## 

        self.fc2 = nn.Linear(in_features=120,out_features=60)
        self.out = nn.Linear(in_features=60,out_features=10)
        # we descrease the number of nodes per layer
    
    def forward(self, t):
        ## Convolution layers (with relu and maxpool)
        t=F.max_pool2d(F.relu(self.conv1(t)),kernel_size=2,stride=2)
        t=F.max_pool2d(F.relu(self.conv2(t)),kernel_size=2,stride=2)

        ## Flatten
        t=t.reshape(-1,12*4*4)

        ## Linear layers
        t=F.relu(self.fc1(t))
        t=F.relu(self.fc2(t))

        ## Output Layer
        t=self.out(t)
        # t=F.softmax(t,dim=1)
        #dont have to mention softmax, cuz categorical crossentropy 
        #select korlei eita selecet hoye jay

        return t

In [7]:
train_set = torchvision.datasets.FashionMNIST(
    root='data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)
train_loader=torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=True)
test_set = torchvision.datasets.FashionMNIST(
    root='data/FashionMNIST',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)
test_loader=torch.utils.data.DataLoader(test_set, batch_size=batch_size)

0it [00:00, ?it/s]

26427392it [00:04, 5643945.21it/s]                              


Extracting data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 36165.72it/s]                           
0it [00:00, ?it/s]

Extracting data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz


4423680it [00:02, 1694880.52it/s]                            
0it [00:00, ?it/s]

Extracting data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 15392.92it/s]            

Extracting data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


## Starting with TensorBoard

In [0]:
tb = SummaryWriter()

network = Network()
images,labels = next(iter(train_loader))
grid=torchvision.utils.make_grid(images)

tb.add_image('images',grid)
tb.add_graph(network, images)
tb.close()

## Training loop

In [0]:
def train(batch_size,lr,shuffle,comment):
  network = Network()
  train_loader=torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=shuffle)
  test_loader=torch.utils.data.DataLoader(test_set, batch_size=batch_size)

  num_epochs = 1

  optimizer = optim.Adam(network.parameters(), lr=lr)

  images,labels = next(iter(train_loader))
  grid=torchvision.utils.make_grid(images)

  ##summary writer will append the comment to the name of the run
  ##so we will be able to recgnize it in tensorboard
  
  #comment=f' batch_size={batch_size} lr={lr}' 
  ## Now RunBuilder can handle arbitrary number of parameters
  
  tb = SummaryWriter(comment=comment)
  tb.add_image('images',grid)
  tb.add_graph(network, images)

  network = network.to('cuda')

  count = 0
  iteration_list = []

  train_loss_list = []
  valid_loss_list = []
  train_accuracy_list = []
  valid_accuracy_list = []

  for epoch in range(num_epochs):

      total_train_loss = 0
      total_train_correct = 0
      total_train = 0

      for i, data in enumerate(train_loader):

          network.train()

          inputs, labels = data
          inputs = inputs.to('cuda')
          labels = labels.to('cuda')

          outputs = network(inputs) # Forward propagation
          loss = F.cross_entropy(outputs, labels) # Calculate softmax and cross entropy loss
          
          optimizer.zero_grad()
          loss.backward()# Calculating gradients
          optimizer.step()# Update parameters
          
          count += 1
          total_train_loss += loss.item()*len(labels)
          trloss=loss.item()
          total_train_correct += get_num_correct(outputs,labels)
          total_train += len(labels)

          # TEST LOSS and ACCURACY
          if count%50 == 0:
              with torch.no_grad():

                  network.eval() #for dropout, batchnorm type things

                  total_valid_correct = 0
                  total_valid = 0
                  total_valid_loss = 0

                  for data in test_loader:

                      images, labels = data
                      images = images.to('cuda')
                      labels = labels.to('cuda')

                      outputs = network(images)
                      loss = F.cross_entropy(outputs, labels)

                      total_valid_loss += loss.item()*len(labels)
                      vlloss=loss.item()
                      total_valid += len(labels)
                      total_valid_correct += get_num_correct(outputs, labels)

              valid_accuracy = 100 * total_valid_correct / float(total_valid)
              train_accuracy = 100 * total_train_correct / float(total_train)

              train_loss_list.append(total_train_loss/float(total_train))
              valid_loss_list.append(total_valid_loss/float(total_valid))
              valid_accuracy_list.append(valid_accuracy)
              train_accuracy_list.append(train_accuracy)
              iteration_list.append(count)

              if count%200 == 0:
                  print('Iteration/Epoch: {}/{}  Train_Loss: {:.4f} Train_Acc: {:.4f}% Test_Loss: {:.4f}  Test_Acc: {:.4f}%'
                        .format(i,epoch+1,train_loss_list[-1],train_accuracy_list[-1],valid_loss_list[-1],valid_accuracy_list[-1]))
      
      tb.add_scalar('Train Loss', total_train_loss, epoch)
      tb.add_scalar('Valid Loss', total_valid_loss, epoch)
      tb.add_scalar('Train Acc', train_accuracy_list[-1], epoch)
      tb.add_scalar('Valid Acc', valid_accuracy_list[-1], epoch)

      ##passing a set of values, to create a histogram
      for name, weight in network.named_parameters():
        tb.add_histogram(name,weight,epoch)
        tb.add_histogram(f'{name}.grad', weight.grad, epoch)

  tb.close()
  print('Finished Training')

**The main power of tensorboard comes when experimenting with multiple runs, and we can compare all those runs side by side. Suppose hyper parameter tuning, we can compare all accuracy and loss graphs with all those runs and find out the best model from these!**

## Hyper-parameter tuning and experimenting

### We can loop over parameter combinations

In [0]:
lr = [0.01,0.001]
batch_size = [10, 100, 1000]
shuffle = [True, False]

In [0]:
for a in lr:
  for b in batch-size:
    train(a,b,True)

### There is a better way

In [0]:
from itertools import product

In [0]:
parameters = dict(
    lr = [0.01,0.001],
    batch_size = [10, 100, 1000],
    shuffle = [True, False]
)

In [0]:
param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001], [10, 100, 1000], [True, False]]

In [0]:
for lr, batch, shuffle in product(*param_values):
  print(lr, batch, shuffle)

0.01 10 True
0.01 10 False
0.01 100 True
0.01 100 False
0.01 1000 True
0.01 1000 False
0.001 10 True
0.001 10 False
0.001 100 True
0.001 100 False
0.001 1000 True
0.001 1000 False


In [0]:
for lr, batch, shuffle in product(*param_values):
  comment=f' batch_size={batch_size} lr={lr}'
  train(batch,lr,shuffle,comment)

Iteration/Epoch: 199/1  Train_Loss: 1.4644 Train_Acc: 44.8500% Test_Loss: 0.9728  Test_Acc: 61.6200%
Iteration/Epoch: 399/1  Train_Loss: 1.1825 Train_Acc: 55.2000% Test_Loss: 0.8515  Test_Acc: 67.5100%
Iteration/Epoch: 599/1  Train_Loss: 1.0539 Train_Acc: 59.8333% Test_Loss: 0.7483  Test_Acc: 71.2000%
Iteration/Epoch: 799/1  Train_Loss: 0.9876 Train_Acc: 62.2250% Test_Loss: 0.7216  Test_Acc: 71.8800%
Iteration/Epoch: 999/1  Train_Loss: 0.9307 Train_Acc: 64.3000% Test_Loss: 0.7657  Test_Acc: 71.1400%
Iteration/Epoch: 1199/1  Train_Loss: 0.8921 Train_Acc: 65.6500% Test_Loss: 0.7269  Test_Acc: 72.7000%
Iteration/Epoch: 1399/1  Train_Loss: 0.8679 Train_Acc: 66.5571% Test_Loss: 0.6752  Test_Acc: 74.4200%
Iteration/Epoch: 1599/1  Train_Loss: 0.8469 Train_Acc: 67.4500% Test_Loss: 0.7168  Test_Acc: 71.1500%
Iteration/Epoch: 1799/1  Train_Loss: 0.8268 Train_Acc: 68.2444% Test_Loss: 0.6348  Test_Acc: 75.6300%
Iteration/Epoch: 1999/1  Train_Loss: 0.8072 Train_Acc: 69.1000% Test_Loss: 0.6602  Test

In [0]:
!ls runs

'Aug26_17-43-15_95eab96116ed batch_size=10 lr=0.01'
'Aug26_17-48-30_95eab96116ed batch_size=10 lr=0.01'
'Aug26_17-53-38_95eab96116ed batch_size=100 lr=0.01'
'Aug26_17-54-04_95eab96116ed batch_size=100 lr=0.01'
'Aug26_17-54-31_95eab96116ed batch_size=1000 lr=0.01'
'Aug26_17-54-41_95eab96116ed batch_size=1000 lr=0.01'
'Aug26_17-54-52_95eab96116ed batch_size=10 lr=0.001'
'Aug26_18-00-03_95eab96116ed batch_size=10 lr=0.001'
'Aug26_18-05-14_95eab96116ed batch_size=100 lr=0.001'
'Aug26_18-05-41_95eab96116ed batch_size=100 lr=0.001'
'Aug26_18-06-07_95eab96116ed batch_size=1000 lr=0.001'
'Aug26_18-06-18_95eab96116ed batch_size=1000 lr=0.001'


## Run Builder Class

In [0]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [0]:
params = OrderedDict(
    lr = [0.01,0.001],
    batch_size = [64, 128]
)

In [0]:
Run = namedtuple('Run',params.keys())

In [0]:
run = []
for v in product(*params.values()):
  run.append(Run(*v)) #take the tuple values as the arguments, not the tuple itself
  
run

[Run(lr=0.01, batch_size=64),
 Run(lr=0.01, batch_size=128),
 Run(lr=0.001, batch_size=64),
 Run(lr=0.001, batch_size=128)]

In [0]:
run[3].lr, run[1].batch_size

(0.001, 128)

In [0]:
class RunBuilder():
  @staticmethod
  def get_runs(params):
    Run = namedtuple('Run',params.keys())
    run = []
    for v in product(*params.values()):
      run.append(Run(*v))
    return run

In [0]:
#### BEFORE ####
# for lr, batch, shuffle in product(*param_values):
#   train(batch,lr,shuffle)
#### AFTER ####
for run in RunBuilder.get_runs(params):
  comment = f'-{run}'
  train(run.batch_size,run.lr,True,comment)

Iteration/Epoch: 199/1  Train_Loss: 0.8366 Train_Acc: 68.2891% Test_Loss: 0.6665  Test_Acc: 75.0600%
Iteration/Epoch: 399/1  Train_Loss: 0.7003 Train_Acc: 73.6211% Test_Loss: 0.5222  Test_Acc: 80.7500%
Iteration/Epoch: 599/1  Train_Loss: 0.6329 Train_Acc: 76.1927% Test_Loss: 0.5206  Test_Acc: 80.4300%
Iteration/Epoch: 799/1  Train_Loss: 0.5877 Train_Acc: 77.9492% Test_Loss: 0.4409  Test_Acc: 83.7000%
Finished Training
Iteration/Epoch: 199/1  Train_Loss: 0.8338 Train_Acc: 67.4609% Test_Loss: 0.6110  Test_Acc: 76.3000%
Iteration/Epoch: 399/1  Train_Loss: 0.6855 Train_Acc: 73.6387% Test_Loss: 0.5014  Test_Acc: 81.2100%
Finished Training
Iteration/Epoch: 199/1  Train_Loss: 1.1056 Train_Acc: 58.7812% Test_Loss: 0.7720  Test_Acc: 69.8100%
Iteration/Epoch: 399/1  Train_Loss: 0.9049 Train_Acc: 66.0586% Test_Loss: 0.6580  Test_Acc: 74.7400%
Iteration/Epoch: 599/1  Train_Loss: 0.8106 Train_Acc: 69.4115% Test_Loss: 0.5771  Test_Acc: 78.0800%
Iteration/Epoch: 799/1  Train_Loss: 0.7472 Train_Acc: 7

## Training loop refactoring and simulataneous hyperparamater testing

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

In [0]:
class RunManager():
  def __init__(self):

    self.epoch_count = 0
    self.epoch_loss = 0
    self.epoch_num_correct = 0
    self.epoch_valid_loss = 0
    self.epoch_valid_num_correct = 0
    self.epoch_start_time = None

    self.run_params = None
    self.run_count = 0
    self.run_data = []
    self.run_start_time = None

    self.network = None
    self.trainloader = None
    self.testloader = None
    self.tb = None
  
  def begin_run(self, run, network, trainloader, testloader):

    self.run_start_time = time.time()

    self.run_params = run
    self.run_count += 1

    self.network = network
    self.trainloader = trainloader
    self.testloader = testloader
    self.tb = SummaryWriter(comment=f'-{run}')

    images, labels = next(iter(self.trainloader))
    images=images.to('cuda')
    grid = torchvision.utils.make_grid(images)

    self.tb.add_image('images', grid)
    self.tb.add_graph(self.network, images)

  def end_run(self):
    self.tb.close()
    self.epoch_count = 0

  def begin_epoch(self):
    self.epoch_start_time = time.time()

    self.epoch_count += 1
    self.epoch_loss = 0
    self.epoch_num_correct = 0
    self.epoch_valid_loss = 0
    self.epoch_valid_num_correct = 0

  def end_epoch(self):

    epoch_duration = time.time() - self.epoch_start_time
    run_duration = time.time() - self.run_start_time

    loss = self.epoch_loss / len(self.trainloader.dataset)
    accuracy = self.epoch_num_correct / len(self.trainloader.dataset)

    valid_loss = self.epoch_valid_loss / len(self.testloader.dataset)
    valid_accuracy = self.epoch_valid_num_correct / len(self.testloader.dataset)

    self.tb.add_scalar('Loss', loss, self.epoch_count)
    self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

    self.tb.add_scalar('Valid Loss', loss, self.epoch_count)
    self.tb.add_scalar('Valid Accuracy', accuracy, self.epoch_count)

    for name, param in self.network.named_parameters():
      self.tb.add_histogram(name, param, self.epoch_count)
      self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)

    results = OrderedDict()
    results["run"] = self.run_count
    results["epoch"] = self.epoch_count
    results['loss'] = loss
    results["accuracy"] = accuracy
    results['valid loss'] = valid_loss
    results["valid accuracy"] = valid_accuracy
    results['epoch duration'] = epoch_duration
    results['run duration'] = run_duration

    for k,v in self.run_params._asdict().items(): results[k] = v
    self.run_data.append(results)

    df = pd.DataFrame.from_dict(self.run_data, orient='columns')

    clear_output(wait=True)
    display(df)

  def track_loss(self, loss):
    self.epoch_loss += loss.item() * self.trainloader.batch_size

  def track_num_correct(self, preds, labels):
    self.epoch_num_correct += self._get_num_correct(preds, labels)

  def track_valid_loss(self, loss):
    self.epoch_valid_loss += loss.item() * self.testloader.batch_size

  def track_valid_num_correct(self, preds, labels):
    self.epoch_valid_num_correct += self._get_num_correct(preds, labels)

  @torch.no_grad()
  def _get_num_correct(self, preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

  def save(self, fileName):
    
    pd.DataFrame.from_dict(
        self.run_data, orient='columns'
    ).to_csv(f'{fileName}.csv')

    with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
        json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [26]:
params = OrderedDict(
    lr = [.01]
    ,batch_size = [1000, 2000]
    ,shuffle = [True]
)
m = RunManager()
for run in RunBuilder.get_runs(params):
  
  network = Network()
  network = network.to('cuda')
  
  train_loader=torch.utils.data.DataLoader(train_set, batch_size=run.batch_size,shuffle=run.shuffle)
  test_loader=torch.utils.data.DataLoader(test_set, batch_size=run.batch_size)
  
  optimizer = optim.Adam(network.parameters(), lr=run.lr)

  m.begin_run(run,network,train_loader,test_loader)
  for epoch in range(5):
    m.begin_epoch()

    ##training
    network.train()
    for data in train_loader:
      inputs, labels = data[0].to('cuda'), data[1].to('cuda')
      outputs = network(inputs)
      loss = F.cross_entropy(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      m.track_loss(loss)
      m.track_num_correct(outputs,labels)
    
    ## validation
    with torch.no_grad():
      network.eval()
      for data in test_loader:
        inputs, labels = data[0].to('cuda'), data[1].to('cuda')
        outputs = network(inputs)
        loss = F.cross_entropy(outputs, labels)

        m.track_valid_loss(loss)
        m.track_valid_num_correct(outputs,labels)

    m.end_epoch()
  m.end_run()
m.save('results')

,run,epoch,loss,accuracy,valid loss,valid accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,0.962823,0.624650,0.619712,0.7521,9.474450,10.087543,0.01,1000,True
1,1,2,0.546383,0.784817,0.518220,0.8043,9.602154,19.853848,0.01,1000,True
2,1,3,0.447019,0.833367,0.439939,0.8372,9.659519,29.650491,0.01,1000,True
3,1,4,0.406291,0.850567,0.396699,0.8529,9.612568,39.406079,0.01,1000,True
4,1,5,0.364422,0.865817,0.381028,0.8649,9.595242,49.148579,0.01,1000,True
5,2,1,1.260493,0.524867,0.732135,0.7315,9.817601,10.743270,0.01,2000,True
6,2,2,0.637478,0.750933,0.572166,0.7745,9.703493,20.589442,0.01,2000,True
7,2,3,0.522696,0.798367,0.512680,0.8065,9.639640,30.369701,0.01,2000,True
8,2,4,0.465717,0.827367,0.454987,0.8349,9.656057,40.173634,0.01,2000,True
9,2,5,0.419567,0.847950,0.421784,0.8468,9.653873,49.971560,0.01,2000,True


#### Code smells in this version of refactoring:
1. many variables with the name epoch. should be extracted into a separate class
2. some to('cuda') here and there. this needs to be specified in one place
3. validation and training loss,acc calculation has different functions. needs to be combined.